In [ ]:
# Import packages.
import cv2
import os
import numpy as np
from PIL import Image
import pickle

BASE_DIR = os.getcwd()    # get the path location of this python file (or the path location to this notebook).
image_dir = os.path.join(BASE_DIR, "images")    # get path location of 'images' file.

In [ ]:
# Intel built-in haar cascade classifiers. Use for detection.
face_cascade = cv2.CascadeClassifier("cascades/data/haarcascade_frontalface_alt2.xml")    # front-of-face cascade.

recognizer = cv2.face.LBPHFaceRecognizer_create()    # creating an LBPH Face Recognizer from CV2.

x_train = []    # initializing an empty list for the NUMPY arrays (i.e. pixel values) that correspond to the specific
# faces within the images.
y_labels = []    # initializing an empty list for image label ID's.
current_id = 0    # initializing label ID's to 0.
labels_ids = {}    # initializing empty dictionary for label-ID (KEY=label, VALUE=ID).

In [ ]:
# Go through EACH image folder within 'images'. Get path of each image & label (name of person in image as well as
# name of the folder that image is located in) + MORE.
for root, dirs, files in os.walk(image_dir):
    # Go through EACH image.
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg"):
            img_path = os.path.join(root, file)    # get path location of image.
            img_label = os.path.basename(root).upper()    # get label of image i.e. name of person the image is showing.
            print(img_label, img_path)

            # If the label of the image is not already in the dictionary (which all won't be to begin with), then assign
            # the value of the 'current_id' to that label in the dictionary. Then, current_id++.
            if img_label not in labels_ids:
                labels_ids[img_label] = current_id
                current_id += 1
            id_ = labels_ids[img_label]    # 'id_' is the value associating to a certain label in the dictionary.

            pil_image = Image.open(img_path).convert("L")    # returns the image corresponding to the path in grayscale.
            size = (300, 300)    # resize all images to this dimension.
            final_img = pil_image.resize(size, Image.ANTIALIAS)
            img_array = np.array(final_img, "uint8")    # turns the image into NUMPY array.

            # Face detection from image array.
            faces = face_cascade.detectMultiScale(img_array, scaleFactor=1.3, minNeighbors=5)
            for (x, y, w, h) in faces:
                roi = img_array[y:y+h, x:x+w]
                x_train.append(roi)    # add to the 'x_train' list the ROI (face), but in a NUMPY array.
                y_labels.append(id_)    # add to the 'y_labels' list the ID's associating to each label.

In [ ]:
# Using Pickle to save the label-ID dictionary in order to use in the Model.
with open("labels.pickle", "wb") as f:
    pickle.dump(labels_ids, f)

In [ ]:
# Training the recognizer with the training data (NUMPY arrays of the ROI's) and the labels (in a NUMPY array).
recognizer.train(x_train, np.array(y_labels))
recognizer.save("training_data.yml")    # saving the trained data file as .yml.